In [1]:
from langchain.document_loaders import TextLoader
from langchain.chat_models import ChatOpenAI
from langchain.prompts.chat import ChatPromptTemplate
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv())

True

# Interface

In [2]:
model = ChatOpenAI()
prompt = ChatPromptTemplate.from_template("tell me a joke about {topic}")
chain = prompt | model

## Stream

In [3]:
for s in chain.stream({"topic": "bears"}):
    print(s.content, end='', flush=True)

Why don't bears wear shoes?

Because they have bear feet!

## Invoke

In [4]:
chain.invoke({"topic": "bears"})

AIMessage(content="Why don't bears wear shoes?\n\nBecause they have bear feet!")

## Batch

In [5]:
chain.batch([{"topic": "bears"}, {"topic": "cats"}], config={"max_concurency": 5})

[AIMessage(content="Why don't bears wear shoes?\n\nBecause they'd rather go bear foot!"),
 AIMessage(content="Sure, here's a cat-themed joke for you:\n\nWhy did the cat sit on the computer?\n\nBecause it wanted to keep an eye on the mouse!")]

# Parallelism

In [6]:
from langchain.schema.runnable import RunnableParallel

chain1 = ChatPromptTemplate.from_template("tell me a joke about {topic}") | model
chain2 = (
    ChatPromptTemplate.from_template("write a short (2 line) poem about {topic}")
    | model
)
combined = RunnableParallel(joke=chain1, poem=chain2)

In [7]:
chain1.invoke({"topic": "bears"})

AIMessage(content="Why don't bears wear shoes?\n\nBecause they have bear feet!")

In [8]:
chain2.invoke({"topic": "bears"})

AIMessage(content="In the forest's embrace, bears dance with might,\nMajestic creatures, painting the night.")

In [9]:
combined.invoke({"topic": "bears"})

{'joke': AIMessage(content="Why don't bears wear shoes? \n\nBecause they already have bear feet!"),
 'poem': AIMessage(content="In the forest's embrace,\nBears roam with gentle grace.")}

## Parallelism on batches

In [10]:
chain1.batch([{"topic": "bears"}, {"topic": "cats"}])

[AIMessage(content="Sure, here's a bear joke for you:\n\nWhy don't bears wear shoes?\n\nBecause they have bear feet!"),
 AIMessage(content="Why don't cats play poker in the wild?\n\nToo many cheetahs!")]

In [11]:
chain2.batch([{"topic": "bears"}, {"topic": "cats"}])

[AIMessage(content='Mighty in the wild, \nBears roam with grace and might.'),
 AIMessage(content='Whiskers gently dance,\nSilent hunters in a trance.')]

In [12]:
combined.batch([{"topic": "bears"}, {"topic": "cats"}])

[{'joke': AIMessage(content="Why don't bears wear shoes?\n\nBecause they have bear feet!"),
  'poem': AIMessage(content="Bears roam wild, nature's might,\nIn their rugged splendor, a majestic sight.")},
 {'joke': AIMessage(content="Why don't cats play poker in the wild?\n\nToo many cheetahs!"),
  'poem': AIMessage(content="Whiskers soft, eyes gleaming bright,\nCats' grace, a mystery in the night.")}]

## Bind runtime args

In [4]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough

prompt = ChatPromptTemplate.from_messages(
    [
        ("system",
        """
        Write out the following equation using algebraic symbols then solve it.
        Use the format\n\nEQUATION:...\nSOLUTION:...\n\n
        """,
        ),
        ("human", "{equation_statement}"),
    ]
)
model = ChatOpenAI(temperature=0)
runnable = (
    {"equation_statement": RunnablePassthrough()} 
    | prompt 
    | model 
    | StrOutputParser()
)

print(runnable.invoke("x raised to the third plus seven equals 12"))

EQUATION: x^3 + 7 = 12

SOLUTION:
Subtracting 7 from both sides of the equation, we get:
x^3 = 12 - 7
x^3 = 5

Taking the cube root of both sides, we find:
x = ∛5

Therefore, the solution to the equation is x = ∛5.


## stop word

In [15]:
runnable = (
    {"equation_statement": RunnablePassthrough()}
    | prompt
    | model.bind(stop="SOLUTION")
    | StrOutputParser()
)
print(runnable.invoke("x raised to the third plus seven equals 12"))

EQUATION: x^3 + 7 = 12




## Attaching OpenAI functions

In [16]:
functions = [
    {
        "name": "solver",
        "description": "Formulates and solves an equation",
        "parameters": {
            "type": "object",
            "properties": {
                "equation": {
                    "type": "string",
                    "description": "The algebraic expression of the equation",
                },
                "solution": {
                    "type": "string",
                    "description": "The solution to the equation",
                },
            },
            "required": ["equation", "solution"],
        },
    }
]

In [17]:
# Need gpt-4 to solve this one correctly
prompt = ChatPromptTemplate.from_messages(
    [
        ("system",
        "Write out the following equation using algebraic symbols then solve it.",
        ),
        ("human", "{equation_statement}"),
    ]
)
model = ChatOpenAI(model="gpt-4", temperature=0).bind(
    function_call={"name": "solver"}, functions=functions
)
runnable = {"equation_statement": RunnablePassthrough()} | prompt | model
runnable.invoke("x raised to the third plus seven equals 12")

AIMessage(content='', additional_kwargs={'function_call': {'name': 'solver', 'arguments': '{\n"equation": "x^3 + 7 = 12",\n"solution": "x = ∛5"\n}'}})

# PromptTemplate + LLM + OutputParser

In [18]:
from langchain.prompts import ChatPromptTemplate
from langchain.chat_models import ChatOpenAI

prompt = ChatPromptTemplate.from_template("tell me a joke about {foo}")
model = ChatOpenAI()
chain = prompt | model

In [19]:
functions = [
    {
        "name": "joke",
        "description": "A joke",
        "parameters": {
            "type": "object",
            "properties": {
                "setup": {"type": "string", "description": "The setup for the joke"},
                "punchline": {
                    "type": "string",
                    "description": "The punchline for the joke",
                },
            },
            "required": ["setup", "punchline"],
        },
    }
]
chain = prompt | model.bind(function_call={"name": "joke"}, functions=functions)

In [20]:
from langchain.output_parsers.openai_functions import JsonOutputFunctionsParser

chain = (
    prompt
    | model.bind(function_call={"name": "joke"}, functions=functions)
    | JsonOutputFunctionsParser()
)

In [21]:
chain.invoke({"foo": "bears"})

{'setup': "Why don't bears wear shoes?",
 'punchline': 'Because they have bear feet!'}

In [22]:
from langchain.output_parsers.openai_functions import JsonKeyOutputFunctionsParser

chain = (
    prompt
    | model.bind(function_call={"name": "joke"}, functions=functions)
    | JsonKeyOutputFunctionsParser(key_name="setup")
)

In [23]:
chain.invoke({"foo": "bears"})

"Why don't bears wear shoes?"